In [ ]:
import pydicom
import redbrick
import os
import pandas as pd
import json
import numpy as np

# Basic setup

In [3]:
org_id = ""
api_key = ""
url = "https://app.redbrickai.com"
taxonomy_name = "X-ray anomaly"
project_name = "X-ray anomaly"

org = redbrick.get_org(org_id=org_id, api_key=api_key, url=url)

# Creating a taxonomy
Taxonomies define what you are annotating in your images. 

In [ ]:
findings = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis"
]

object_types = []
for i, finding in enumerate(findings): 
    object_types += [
        {
            "category": finding, 
            "labelType": "BBOX", 
            "classId": i + 1
        }
    ]

study_classify = [
    {
        "name": "Finding", 
        "attrId": 1, 
        "attrType": "BOOL"
    }
]

org.create_taxonomy(
    name=taxonomy_name,
    object_types=object_types, 
    study_classify=study_classify
)

# Prepare dataset upload

In [5]:
files = filter(lambda x: x.endswith(".dicom"), os.listdir("data"))
df = pd.read_csv("train.csv")
items = []

for file in files: 
    id = file.replace(".dicom", "")
    
    # Get row for image id
    row = df.loc[df['image_id'] == id].iloc[0]

    class_name = row["class_name"]
    series = {
        "items": os.path.join("data", file), 
    }
    
    # Add bounding box if it exists
    if class_name != "No finding": 
        img = pydicom.dcmread(os.path.join("data", file))
        height, width = img.pixel_array.shape

        bbox = {
            "pointTopLeft": {
                "xNorm": row["x_min"] / width, 
                "yNorm": row["y_min"] / height,
            },
            "hNorm": (row["x_max"] - row["x_min"]) / width, 
            "wNorm": (row["y_max"] - row["y_min"]) / height, 
            "category": class_name
        }
        series["boundingBoxes"] = [bbox]


    items += [
        {
            "name": class_name + "_" + id, 
            "series": [series],
            "priority": np.random.uniform(0, 1),
            "classification": {
                "attributes": {
                    "Finding": False if class_name == "No finding" else True
                }
            }
        }
    ]

with open("items.json", "w+") as file:
    json.dump(items, file, indent=2)

# Create project and upload data

In [6]:
project = org.create_project_advanced(
    name=project_name, 
    taxonomy_name=taxonomy_name, 
    stages=[
        redbrick.LabelStage(
            stage_name="Label", 
            config=redbrick.LabelStage.Config(auto_assignment=False, show_uploaded_annotations=False),
            on_submit="Tie_breaker"
        ),
        redbrick.ReviewStage(
            stage_name="Tie_breaker", 
            config=redbrick.ReviewStage.Config(auto_assignment=False),
            on_accept="Internal",
            on_reject="Label"
        ),
        redbrick.ReviewStage(
            stage_name="Internal", 
            config=redbrick.ReviewStage.Config(auto_assignment=False),
            on_reject="Label"
        )
    ]
)

In [ ]:
tasks = project.upload.create_datapoints(
    storage_id=redbrick.StorageMethod.REDBRICK, 
    points=items
)